## Сентимент-анализ отзывов на товары (простая версия)

Классифицируйте отзывы по тональности

В [этом соревновании](https://inclass.kaggle.com/c/product-reviews-sentiment-analysis-light) вам предстоит прогнозировать по тексту отзыва его тональность: 1 - позитивная, 0 - негативная. В отличие от усложненной версии задачи, здесь вам не требуется самостоятельно собирать обучающую выборку - она есть в предоставляемых вам данных.

In [1]:
import pandas as pd
%matplotlib inline

In [2]:
train = pd.read_table('products_sentiment_train.tsv')
X_test = pd.read_table('products_sentiment_test.tsv')
y_test = pd.read_csv('products_sentiment_sample_submission.csv')

In [3]:
X_test.drop('Id', axis=1, inplace=True)
y_test.drop('Id', axis=1, inplace=True)

X_train = train.drop('y', axis=1)
y_train = train.drop('text', axis=1)

#train['y'] = train['y'].replace(to_replace=[0, 1], value=['neg', 'pos'])
#y_test['y'] = y_test['y'].replace(to_replace=[0, 1], value=['neg', 'pos'])
train.head()

,text,y
0,"2. take around 10,000 640x480 pictures .",1
1,i downloaded a trial version of computer assoc...,1
2,the wrt54g plus the hga7t is a perfect solutio...,1
3,i dont especially like how music files are uns...,0
4,i was using the cheapie pail ... and it worked...,1


In [4]:
#y_train.head()

In [5]:
#X_test.head()

In [6]:
#y_test.head(6)

In [7]:
print train.shape, X_test.shape

(2000, 2) (500, 1)


In [8]:
print train['text'][3]

i dont especially like how music files are unstructured ; basically they are just dumped into one folder with no organization , like you might have in windows explorer folders and subfolders .


[Tutorial: Simple Text Classification with Python and TextBlob](http://stevenloria.com/how-to-build-a-text-classification-system-with-python-and-textblob/)

In [9]:
from textblob.classifiers import NaiveBayesClassifier

X_test.insert(1, 'y', y_test['y'].values)

Train = [(line[0], line[1]) for line in train.values]
Test = [(line[0], line[1]) for line in X_test.values]

In [10]:
print Test[3]

('6 . play gameboy color games on it with goboy .', 1L)


In [11]:
%time
clf = NaiveBayesClassifier(Train)

Wall time: 0 ns


In [12]:
clf.classify("mine was 2 weeks old and i chucked it in the trash , where it belongs .")

1L

In [13]:
prec = []
for i, line in enumerate(X_test['text'].values.tolist()):
    prec.append(y_test.ix[i, 'y'] == clf.classify(line))
print sum(prec) / float(len(prec))

0.454


In [14]:
from sklearn.metrics import f1_score
import numpy as np
pred = []
for line in X_test['text'].values.tolist():
    pred.append(clf.classify(line))
    
F1_score = f1_score(np.array(y_test.ix[:, 'y']), pred)
print F1_score

0.553191489362


Векторизуем тексты рецензий:

In [15]:
texts_train = train.text.values
texts_test = X_test.text.values

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(sublinear_tf=True, use_idf=True)
X_train = vect.fit_transform(texts_train)
X_test = vect.transform(texts_test)

### Логистическая регрессия

Настроим на векторизованных данных логистическую регрессию и посчитаем AUC:

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
clf = LogisticRegression()
clf.fit(X_train, y_train)
print metrics.accuracy_score(y_test, clf.predict(X_test))
print metrics.roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])

0.472
0.46264


C:\Users\Insassin\Anaconda2\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [18]:
from sklearn.decomposition import TruncatedSVD
tsvd = TruncatedSVD(n_components=100)
X_train_pca = tsvd.fit_transform(X_train)
X_test_pca = tsvd.transform(X_test)

In [19]:
clf = LogisticRegression()
clf.fit(X_train_pca, y_train)
print metrics.accuracy_score(y_test, clf.predict(X_test_pca))
print metrics.roc_auc_score(y_test, clf.predict_proba(X_test_pca)[:, 1])

0.49
0.465408


In [20]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train_pca, y_train)
print metrics.accuracy_score(y_test, clf.predict(X_test_pca))
print metrics.roc_auc_score(y_test, clf.predict_proba(X_test_pca)[:, 1])

C:\Users\Insassin\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  app.launch_new_instance()


0.5
0.48476
